<a href="https://colab.research.google.com/github/MWFK/NLP-Semantic-Similarity/blob/main/ClinicalTrials/ctdt_FE_NLTK_Cosine_SoftCosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libs


In [ ]:
# Python libs to manipulate dataframes and arrays
import pandas as pd
import numpy as np

# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

# To get the word vectors, you need a word embedding model. Let’s download the FastText model using gensim’s downloader api.
import gensim
print(gensim.__version__)

# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess

3.6.0


### Data

In [ ]:
# Define the documents
doc_trump    = "Mr. Trump became president after winning the political election. Though he lost the support of some republican friends, Trump is friends with President Putin"
doc_election = "President Trump says Putin had no political interference in the election outcome. He says it was a witchhunt by political parties. He claimed President Putin is a friend who had nothing to do with the election"
doc_putin    = "Post elections, Vladimir Putin became President of Russia. President Putin had served as the Prime Minister earlier in his political career"
documents    = [doc_trump, doc_election, doc_putin]

### Modeling using Cosine as a metric

To compute the cosine similarity, you need the word count of the words in each document. The CountVectorizer or the TfidfVectorizer from scikit learn lets us compute this. The output of this comes as a sparse_matrix. On this, am optionally converting it to a pandas dataframe to see the word frequencies in a tabular format.

Even better, I could have used the TfidfVectorizer() instead of CountVectorizer(), because it would have downweighted words that occur frequently across docuemnts. Then, use cosine_similarity() to get the final output. It can take the document term matri as a pandas dataframe as well as a sparse matrix as inputs.

In [ ]:
# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
sparse_matrix    = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), index=['doc_trump', 'doc_election', 'doc_putin'])
df

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,career,claimed,earlier,election,elections,friend,friends,interference,lost,minister,mr,outcome,parties,political,post,president,prime,putin,republican,russia,says,served,support,trump,vladimir,winning,witchhunt
doc_trump,0,0,0,1,0,0,2,0,1,0,1,0,0,1,0,2,0,1,1,0,0,0,1,2,0,1,0
doc_election,0,1,0,2,0,1,0,1,0,0,0,1,1,2,0,2,0,2,0,0,2,0,0,1,0,0,1
doc_putin,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,2,1,2,0,1,0,1,0,0,1,0,0


In [ ]:
print(cosine_similarity(df, df))

[[1.         0.51639778 0.36893239]
 [0.51639778 1.         0.45360921]
 [0.36893239 0.45360921 1.        ]]


### Modeling using SoftCosine as a metric

Suppose if you have another set of documents on a completely different topic, say ‘food’, you want a similarity metric that gives higher scores for documents belonging to the same topic and lower scores when comparing docs from different topics. In such case, we need to consider the semantic meaning should be considered. That is, words similar in meaning should be treated as similar. For Example, ‘President’ vs ‘Prime minister’, ‘Food’ vs ‘Dish’, ‘Hi’ vs ‘Hello’ should be considered similar. For this, converting the words into respective word vectors, and then, computing the similarities can address this problem.

In [ ]:
# Define the documents
doc_soup = "Soup is a primarily liquid food, generally served warm or hot (but may be cool or cold), that is made by combining ingredients of meat or vegetables with stock, juice, water, or another liquid. "
doc_noodles = "Noodles are a staple food in many cultures. They are made from unleavened dough which is stretched, extruded, or rolled flat and cut into one of a variety of shapes."
doc_dosa = "Dosa is a type of pancake from the Indian subcontinent, made from a fermented batter. It is somewhat similar to a crepe in appearance. Its main ingredients are rice and black gram."
documents = [doc_trump, doc_election, doc_putin, doc_soup, doc_noodles, doc_dosa]

In [ ]:
%%time
# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded
CPU times: user 9min 36s, sys: 27 s, total: 10min 3s
Wall time: 11min 35s


In [ ]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sent_1 = dictionary.doc2bow(simple_preprocess(doc_trump))
sent_2 = dictionary.doc2bow(simple_preprocess(doc_election))
sent_3 = dictionary.doc2bow(simple_preprocess(doc_putin))
sent_4 = dictionary.doc2bow(simple_preprocess(doc_soup))
sent_5 = dictionary.doc2bow(simple_preprocess(doc_noodles))
sent_6 = dictionary.doc2bow(simple_preprocess(doc_dosa))

sentences = [sent_1, sent_2, sent_3, sent_4, sent_5, sent_6]

In [ ]:
# Compute soft cosine similarity for two sentences
print(softcossim(sent_1, sent_2, similarity_matrix))

0.5885144994929364


In [ ]:
# Compute soft cosine similarity for all sentences
def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

create_soft_cossim_matrix(sentences)

,0,1,2,3,4,5
0,1.00,0.59,0.56,0.28,0.34,0.40
1,0.59,1.00,0.56,0.23,0.33,0.45
2,0.56,0.56,1.00,0.19,0.25,0.36
3,0.28,0.23,0.19,1.00,0.50,0.38
4,0.34,0.33,0.25,0.50,1.00,0.56
5,0.40,0.45,0.36,0.38,0.56,1.00


# Feature Engineering 1
CounVectoriser and TFIDF on the whole InclusionCriterias

In [ ]:
# Python libs to manipulate dataframes and arrays
import pandas as pd
import numpy as np

# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity


def get_data():

  # Download Clinical Trials data
  print('Downloading Clinical Trials Data')
  ct_dt = pd.read_csv(r'https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/Batches_0.csv', sep=',', engine='python', encoding="utf-8")
  for btch in range(1, 4):
      url = 'https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/Batches_' +str(btch)+ '.csv'
      tmp = pd.read_csv(url, sep=',', engine='python', encoding="ISO-8859-1")
      ct_dt = ct_dt.append(tmp, ignore_index=True)
  ct_dt['AllLocation'] = ct_dt['LocationCity'].str.lower().map(str) + ' | ' + ct_dt['LocationState'].str.lower().map(str) + ' | ' + ct_dt['LocationCountry'].str.lower().map(str)
  print('Clinical Trials Data: ',ct_dt.shape, '\n')

  # Download User input data
  print('Downloading Test data')
  test = pd.read_csv('https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/TestData.csv', sep=';', engine='python', encoding = "utf-8", skiprows=[0], names=['PatientID','ConditionOrDisease','Age','Gender','LocationCountry','TravelDistance','InclusionCriteria'])
  print('Test Data: ', test.shape)

  return ct_dt, test

ctdt, test = get_data()


def data_processing(ct_dt):

  print('Data dimensions before Filtering : ', ct_dt.shape, '\n')

  ### Filtering by Age ###
  print('Filtering by Age...')
  tmp = ct_dt[ct_dt.iloc[:,13] <= test.iloc[:1,2][0]]               # compare numerics
  tmp = tmp[tmp.iloc[:,13].str.find(test.iloc[:1,2][0][-5:]) != -1] # Detect the Year/Month
  print('Data dimensions: ', tmp.shape, '\n')

  ### Filtering by Gender ###
  print('Filtering by Gender...')
  tmp = tmp[(tmp.iloc[:,12] == test.iloc[:1,3][0]) | (tmp.iloc[:,12] == 'All')] 
  print('Data dimensions: ', tmp.shape, '\n')

  ### Filtering by Travel Distance ###
  print('Filtering by Travel Distance...')
  tmp = tmp[tmp.iloc[:,20].str.find(test.iloc[:1,5][0].lower()) != -1] 
  print('Data dimensions: ', tmp.shape, '\n')

  return tmp

filtered = data_processing(ct_dt)

Clinical Trials Data:  (10152, 21) 

Test Data:  (7, 7)
Data dimensions before Filtering :  (10152, 21) 

Filtering by Age...
Data dimensions:  (9517, 21) 

Filtering by Gender...
Data dimensions:  (9403, 21) 

Filtering by Travel Distance...
Data dimensions:  (645, 21) 



In [ ]:
%%time

#[1] Data
ctdt['InclusionCriteria']      = ctdt['InclusionCriteria'].fillna(' ') # the whole dataset
filtered['InclusionCriteria']  = filtered['InclusionCriteria'].fillna(' ')   # the subset of the data that we'll score
test0                          = test.iloc[:,6].fillna(' ')            # the user input

'''
TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present in the corpus but also provides the importance of the words. 
We can then remove the words that are less important for analysis, hence making the model building less complex by reducing the input dimensions.
TFIDF is based on the logic that words that are too abundant in a corpus and words that are too rare are both not statistically important for finding a pattern. 
The Logarithmic factor in tfidf mathematically penalizes the words that are too abundant or too rare in the corpus by giving them low tfidf scores.
To train a model on the actual linguistic relationship of the words, there are two other word embedding techniques widely used in NLP, they are "word2vec" and "Glove". 
'''

#[2] Instantiating Vectorization and Feature Engineerings methods
cv  = CountVectorizer(encoding='utf-8', decode_error='ignore', strip_accents='ascii', lowercase=True, stop_words='english', analyzer='word', ngram_range=(2, 2), max_features=100000)
tf = TfidfTransformer(use_idf=True)


#[3] Processing the whole dataset
cv_fitted_ctdt = cv.fit(ctdt['InclusionCriteria'])
cv_trans_ctdt  = cv_fitted_ctdt.transform(ct_dt['InclusionCriteria']) 

tf_fitted_ctdt = tf.fit(cv_trans_ctdt)
tf_trans_ctdt  = tf_fitted_ctdt.transform(cv_trans_ctdt)


#[4] Processing the subset of the data that we'll score 
cv_filtered = cv_fitted_ctdt.transform(filtered['InclusionCriteria'])
tf_filtered = tf_fitted_ctdt.transform(cv_filtered)


#[5] Processing the user input
cv_test0 = cv_fitted_ctdt.transform(test0)
tf_test0 = tf_fitted_ctdt.transform(cv_test0)

CPU times: user 5.97 s, sys: 39.3 ms, total: 6.01 s
Wall time: 6 s


In [ ]:
# for index, row in test.iterrows():
#     #print(index, row['InclusionCriteria'])
#     print('Processing the user input: ', index)
#     test_row = test.iloc[index:index+1,6].fillna(' ') 
#     cv_row = cv_fitted_ctdt.transform(test_row)
#     tf_row = tf_fitted_ctdt.transform(cv_row)

Processing the user input:  0
Processing the user input:  1
Processing the user input:  2
Processing the user input:  3
Processing the user input:  4
Processing the user input:  5
Processing the user input:  6


In [ ]:
tmp['Similarity'] = pd.Series(cosine_similarity(tf_test0, tf_filtered)[0]).values

ctdt_filtered = ctdt
ctdt_filtered['Similarity'] = 0
print('Dataset shape: ', ctdt_filtered.shape)
print('Filtered Dataset shape: ', filtered.shape)

# Take off the existing NCTId from the whole dataset then add the filter data to 
ctdt_filterred = ctdt_filtered[~ctdt_filtered['NCTId'].isin(filtered['NCTId'])]
ctdt_filterred = ctdt_filtered.append(filtered, ignore_index=True)

print('\nScore > 0 ', ctdt_filtered[ctdt_filtered['Similarity']>0].shape)
print('Score > 0.20', ctdt_filtered[ctdt_filtered['Similarity']>0.2].shape)
print('Score > 0.25', ctdt_filtered[ctdt_filtered['Similarity']>0.25].shape)
print('Score > 0.30', ctdt_filtered[ctdt_filtered['Similarity']>0.3].shape)

ctdt_filtered['Similarity'] = ctdt_filtered['Similarity'].apply(lambda score: score if score>0 else 0) # if score>0.25
ctdt_filtered = ctdt_filtered.sort_values(by=['Similarity'], ascending=False)

Dataset shape:  (10152, 22)
Filtered Dataset shape:  (645, 21)

Score > 0  (0, 22)
Score > 0.20 (0, 22)
Score > 0.25 (0, 22)
Score > 0.30 (0, 22)


In [ ]:
def get_results(tf_test0, tf_filtered, filtered, ctdt):
  tmp['Similarity'] = pd.Series(cosine_similarity(tf_test0, tf_filtered)[0]).values

  ctdt_filtered = ctdt
  ctdt_filtered['Similarity'] = 0
  print('Dataset shape: ', ctdt_filtered.shape)
  print('Filtered Dataset shape: ', filtered.shape)

  # Take off the existing NCTId from the whole dataset then add the filter data to 
  ctdt_filtered = ctdt_filtered[~ctdt_filterred['NCTId'].isin(filtered['NCTId'])]
  ctdt_filtered = ctdt_filtered.append(filtered, ignore_index=True)

  print('\nScore > 0 ', ctdt_filtered[ctdt_filtered['Similarity']>0].shape)
  print('Score > 0.20', ctdt_filtered[ctdt_filtered['Similarity']>0.2].shape)
  print('Score > 0.25', ctdt_filtered[ctdt_filtered['Similarity']>0.25].shape)
  print('Score > 0.30', ctdt_filtered[ctdt_filtered['Similarity']>0.3].shape)

  ctdt_filtered['Similarity'] = ctdt_filtered['Similarity'].apply(lambda score: score if score>0.1 else 0) # if score>0.25
  ctdt_filtered = ctdt_filtered.sort_values(by=['Similarity'], ascending=False)

  return ctdt_filterred

# the data have been vectorized
get_results(tf_test0, tf_filtered, filtered, ctdt).head(2)

Dataset shape:  (10152, 22)
Filtered Dataset shape:  (645, 21)

Score > 0  (0, 22)
Score > 0.20 (0, 22)
Score > 0.25 (0, 22)
Score > 0.30 (0, 22)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


,Rank,NCTId,OrgFullName,OfficialTitle,OverallStatus,Keyword,DetailedDescription,Condition,EligibilityCriteria,InclusionCriteria,ExclusionCriteria,HealthyVolunteers,Gender,MinimumAge,StudyPopulation,LocationFacility,LocationCity,LocationState,LocationZip,LocationCountry,AllLocation,Similarity
0,1,NCT03581708,Guangdong Provincial People's Hospital,Real-world Study of the Incidence and Risk Fac...,Not yet recruiting,lung cancer|Venous Thromboembolism,VTE has high incidence in lung cancer and incr...,Lung Neoplasms|Venous Thromboembolism,Inclusion Criteria:||Age ≥ 18 years at the tim...,Age ≥ 18 years at the time of screening.|Easte...,History of another primary malignancy except f...,No,All,18 Years,Patients diagnosed with advanced staged lung c...,Guangdong General Hospital,Guangzhou,Guagndong,510080,China,guangzhou | guagndong | china,0.0
1,2,NCT01130285,University of Toledo Health Science Campus,Validation of a Multi-gene Test for Lung Cance...,"Active, not recruiting",Lung Cancer,"Because more than 160,000 individuals die of l...",Lung Cancer,Inclusion Criteria:||20 or more pack year smok...,20 or more pack year smoking history|clinical ...,Lung Cancer within 3 months after the date of ...,Accepts Healthy Volunteers,All,50 Years,The study population will consist of subjects ...,National Jewish Health|University of Michigan|...,Denver|Ann Arbor|Detroit|Rochester|Cleveland|C...,Colorado|Michigan|Michigan|Minnesota|Ohio|Ohio...,80206|48109|48202|55905|44195|43221|43606|4360...,United States|United States|United States|Unit...,denver|ann arbor|detroit|rochester|cleveland|c...,0.0


In [ ]:
print(ct_dt_tmp['InclusionCriteria'][0])
print(*test0)

'Age ≥ 18 years at the time of screening.|Eastern Cooperative Oncology Group performance status of ≤ 2.|Written informed consent obtained from the patient.|Histologically and cytologically documented Stage 3B-4 lung cancer (according to Version 8 of the International Association for the Study of Lung Cancer Staging system).|Patients with stage 1 to 3, who undergo radical therapy with disease free survival (DFS) >12 months.|Willingness and ability to comply with scheduled visits and other study procedures.||'

### Feature Engineering 2
CountVectorizer and TFIDF on the filtred subset of the InclusionCriterias

In [ ]:
# Python libs to manipulate dataframes and arrays
import pandas as pd
import numpy as np

# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

# To get the word vectors, you need a word embedding model. Let’s download the FastText model using gensim’s downloader api.
import gensim
print(gensim.__version__)

# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess


def get_data():

  # Download Clinical Trials data
  print('Downloading Clinical Trials Data')
  ct_dt = pd.read_csv(r'https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/Batches_0.csv', sep=',', engine='python', encoding="utf-8")
  for btch in range(1, 4):
      url = 'https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/Batches_' +str(btch)+ '.csv'
      tmp = pd.read_csv(url, sep=',', engine='python', encoding="ISO-8859-1")
      ct_dt = ct_dt.append(tmp, ignore_index=True)
  ct_dt['AllLocation'] = ct_dt['LocationCity'].str.lower().map(str) + ' | ' + ct_dt['LocationState'].str.lower().map(str) + ' | ' + ct_dt['LocationCountry'].str.lower().map(str)
  print('Clinical Trials Data: ',ct_dt.shape, '\n')

  # Download User input data
  print('Downloading Test data')
  test = pd.read_csv('https://raw.githubusercontent.com/MWFK/NLP-Semantic-Similarity/main/ClinicalTrials/Data/TestData.csv', sep=';', engine='python', encoding = "utf-8", skiprows=[0], names=['PatientID','ConditionOrDisease','Age','Gender','LocationCountry','TravelDistance','InclusionCriteria'])
  print('Test Data: ', test.shape)

  return ct_dt, test

ct_dt, test = get_data()


def data_processing(ct_dt):

  print('Data dimensions before Filtering : ', ct_dt.shape, '\n')

  ### Filtering by Age ###
  print('Filtering by Age...')
  tmp = ct_dt[ct_dt.iloc[:,13] <= test.iloc[:1,2][0]]               # compare numerics
  tmp = tmp[tmp.iloc[:,13].str.find(test.iloc[:1,2][0][-5:]) != -1] # Detect the Year/Month
  print('Data dimensions: ', tmp.shape, '\n')

  ### Filtering by Gender ###
  print('Filtering by Gender...')
  tmp = tmp[(tmp.iloc[:,12] == test.iloc[:1,3][0]) | (tmp.iloc[:,12] == 'All')] 
  print('Data dimensions: ', tmp.shape, '\n')

  ### Filtering by Travel Distance ###
  print('Filtering by Travel Distance...')
  tmp = tmp[tmp.iloc[:,20].str.find(test.iloc[:1,5][0].lower()) != -1] 
  print('Data dimensions: ', tmp.shape, '\n')

  return tmp

tmp = data_processing(ct_dt)

3.6.0
Clinical Trials Data:  (10152, 21) 

Test Data:  (7, 7)
Data dimensions before Filtering :  (10152, 21) 

Filtering by Age...
Data dimensions:  (9517, 21) 

Filtering by Gender...
Data dimensions:  (9403, 21) 

Filtering by Travel Distance...
Data dimensions:  (645, 21) 



In [ ]:
%%time

#[1] Data
ct_dt['InclusionCriteria'] = ct_dt['InclusionCriteria'].fillna(' ') # the whole dataset
tmp['InclusionCriteria']   = tmp['InclusionCriteria'].fillna(' ')   # the subset of the data that we'll score
test0                      = test.iloc[:1,6].fillna(' ')            # the user input



#[2] Instantiating Vectorization and Feature Engineerings methods
cv  = CountVectorizer(encoding='utf-8', decode_error='ignore', strip_accents='ascii', lowercase=True, stop_words='english', analyzer='word', ngram_range=(2, 2), max_features=100000)
tf = TfidfTransformer(use_idf=True)


#[3] Processing the whole dataset
cv_fitted_tmp = cv.fit(tmp['InclusionCriteria'])
cv_trans_tmp  = cv_fitted_tmp.transform(tmp['InclusionCriteria']) 

tf_fitted_tmp = tf.fit(cv_trans_tmp)
tf_trans_tmp  = tf_fitted_tmp.transform(cv_trans_tmp)


#[4] Processing the user input
cv_test0 = cv_fitted_tmp.transform(test0)
tf_test0 = tf_fitted_tmp.transform(cv_test0)

CPU times: user 725 ms, sys: 8.36 ms, total: 734 ms
Wall time: 762 ms


In [ ]:
tmp['Similarity'] = pd.Series(cosine_similarity(tf_test0, tf_trans_tmp)[0]).values

ct_dt_tmp = ct_dt
ct_dt_tmp['Similarity'] = 0
print(ct_dt_tmp.shape)
print(tmp.shape)

ct_dt_tmp = ct_dt_tmp[~ct_dt_tmp['NCTId'].isin(tmp['NCTId'])]
print(ct_dt_tmp.shape)
ct_dt_tmp = ct_dt_tmp.append(tmp, ignore_index=True)

print(ct_dt_tmp[ct_dt_tmp['Similarity']>0.1].shape)
print(ct_dt_tmp[ct_dt_tmp['Similarity']>0.2].shape)
print(ct_dt_tmp[ct_dt_tmp['Similarity']>0.25].shape)
print(ct_dt_tmp[ct_dt_tmp['Similarity']>0.3].shape)

ct_dt_tmp['Similarity'] = ct_dt_tmp['Similarity'].apply(lambda score: score if score>0 else 0) # if score>0.25
ct_dt_tmp = ct_dt_tmp.sort_values(by=['Similarity'], ascending=False)

(10152, 22)
(645, 22)
(9507, 22)
(14, 22)
(1, 22)
(0, 22)
(0, 22)


In [ ]:
ct_dt_tmp['InclusionCriteria'][0]

'Age ≥ 18 years at the time of screening.|Eastern Cooperative Oncology Group performance status of ≤ 2.|Written informed consent obtained from the patient.|Histologically and cytologically documented Stage 3B-4 lung cancer (according to Version 8 of the International Association for the Study of Lung Cancer Staging system).|Patients with stage 1 to 3, who undergo radical therapy with disease free survival (DFS) >12 months.|Willingness and ability to comply with scheduled visits and other study procedures.||'

In [ ]:
ct_dt_tmp['InclusionCriteria'][0]
print(*test0)

Histologically diagnosed with metastatic non-small cell lung cancer in 2018 | Initially treated with pertuzumab but relapsed | His performance status is ECOG 1 or KPS 90 | His blood and liver function analysis show normal | No other indications like HIV, HCV, HBV | No allergies | Life expectancy over 6 months | No mental disabilities.
